In [2]:
import numpy as np
import maxflow
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import cv2

#### Calculate $\beta$
 - Beta is the smoothness term for the boundary between bg and fg
 - When beta is 1, the boundary is perfectly smooth
 - beta > 0 is preferred to relax constraints on hard boundary b/w pixels
 
 $$
     \beta = \frac{1}{(2*\text{E}(\sqrt{||pixel_{i} - pixel{j}||})}
 $$
 
 Here, $\text{E}$ is the average over all pairs of neighbouring pixels in the image. which are $4wh -3h -3w +2$ in number.

In [3]:
class GrabCut:
    def __init__(self, img, gamma, n_components, max_iter, bgRect, mask, n_iters):
        #  x_start, x_end, y_start, y_end is what we should get after unpacking bgRect
        self.img = img.astype(np.float32)
        self.height, self.width =  list(img.shape)[:2]
        self.k = n_components
        self.gamma = gamma
        self.n_iters = n_iters
        self.bgRect = bgRect
        self.mask = mask
        self.max_iters = max_iter

        # define flags
        self.BG = 0	# confirmed bg
        self.FG = 1	# confirmed fg
        self.PR_BG = 2	# probable bg
        self.PR_FG = 3	# probable fg
        
        # params
        self.beta = 0
        
        self.set_beta() # sets beta
        self.init_graph() # initializes the graph    
        self.set_graph_weights() # sets the weights of the graph
        self.init_trimap(mask) # gets the trimap
        self.add_terminal_edges() # adds the terminal edges


    def set_beta(self):
        beta = 0
        # calculates average over an image sample for (z_m-z_n)^2 for 
        self._left_diff = self.img[:, 1:] - self.img[:, :-1] # Left-difference
        self._upleft_diff = self.img[1:, 1:] - self.img[:-1, :-1] # Up-Left difference
        self._up_diff = self.img[1:, :] - self.img[:-1, :] # Up-difference
        self._upright_diff = self.img[1:, :-1] - self.img[:-1, 1:] # Up-Right difference
        # beta is as described in the paper
        beta = (self._left_diff*self._left_diff).sum() + (self._upleft_diff*self._upleft_diff).sum() \
            + (self._up_diff*self._up_diff).sum() + (self._upright_diff*self._upright_diff).sum() # According to the formula
        self.beta = 1/(2*beta/(4*self.width*self.height - 3*self.width - 3*self.height + 2))
        # 4*self.width*self.height - 3*self.width - 3*self.height + 2 is the number of pairs of neighbouring pixels in the image

    def init_graph(self):
        self.graph = maxflow.Graph[float]()
        self.pixels = self.graph.add_grid_nodes((self.height, self.width))
        

    def set_graph_weights(self):
        for j in range(self.height):
            for i in range(self.width):
                curr_pixel = self.pixels[j, i] # src

    
    